# Filtre des réponses des utilisateurs pour proposer des festivals qui correspondent

**Ce qui nous intéresse dans cette partie est d'utiliser les réponses des utilisateurs pour déterminer des festivals qui correspondent aux attentes.**
<br>**Pour ce faire, nous utilisons les réponses provenant de question.py et envoyons les festivals dans app.py**

## I. Récupérer les informations

### I.1. Réponses de l'utilisateur

In [2]:
import json

# Charger les données utilisateur collectées
try:
    with open("user_data.json", "r") as json_file:
        user_data = json.load(json_file)
        print("Données utilisateur récupérées :", user_data)
except FileNotFoundError:
    print("Aucune donnée utilisateur n'a été collectée. Lancez 'questions.py' pour collecter les données.")

Données utilisateur récupérées : {'name': 'Neau Arthur', 'email': 'neau.arthur@sfr.fr', 'address': '14 Rue  Laurent Vibert 13090 Aix-en-Provence', 'coordinates': [43.533113, 5.43189], 'distance_max': 500, 'types': ['Musique'], 'genres musique': ['Musiques du monde'], 'genres spectacles vivants': [], 'genres cinéma et audiovisuel': [], 'genres arts visuels et numériques': [], 'genres livre et littérature': [], 'dates': ['2025-01-01', '2025-06-20'], 'budget': [20, 100], 'accessible': False}


In [22]:
user_data = {'name': 'Neau Arthur', 'email': 'neau.arthur@sfr.fr', 'address': '14 Rue Laurent Vibert, 13090 Aix-en-Provence, France', 'coordinates': [43.9352448339, 4.84071572505], 'distance_max': 500, 'types': ['Musique'], 'genres musique': ['Musiques du monde', 'Musiques électroniques'], 'genres spectacles vivants': [], 'genres cinéma et audiovisuel': [], 'genres arts visuels et numériques': [], 'genres livre et littérature': [], 'dates': ['2025-01-01', '2025-06-20'], 'budget': [20, 100], 'accessible': False}


### I.2. Base de données des festivals

In [13]:
# Importer le fichier comme un module
from Nettoyage_base_donnees import df
df_a_filtrer = df
print(df_a_filtrer.head())

            Nom du festival Envergure territoriale  \
0          Festival andalou              Régionale   
1  Festival Rêves d'enfants                    NaN   
3                 Pharaonic                    NaN   
4           Sarcus Festival                    NaN   
5          Jazz in Fougères                    NaN   

  Région principale de déroulement Département principal de déroulement  \
0       Provence-Alpes-Côte d'Azur                             Vaucluse   
1       Provence-Alpes-Côte d'Azur                         Hautes-Alpes   
3             Auvergne-Rhône-Alpes                               Savoie   
4              Centre-Val de Loire                       Indre-et-Loire   
5                         Bretagne                      Ille-et-Vilaine   

  Commune principale de déroulement       Site internet du festival  \
0                           Avignon  https://lefestivalandalou.com/   
1                          Briançon         www.serre-chevalier.com   
3          

## II. Filtrer les festivals

In [27]:
from geopy.distance import geodesic
import pandas as pd

# Liste pour stocker les festivals correspondant
filtered_festivals1 = []
filtered_festivals2 = []
filtered_festivals3 = []

# 1. Vérifier la distance
for _, row in df_a_filtrer.iterrows():
    if isinstance(row['Géocodage xy'], str):
        try:
            lat, lon = map(float, row['Géocodage xy'].split(','))
            distance = geodesic(user_data['coordinates'], (lat, lon)).km
            if distance < user_data['distance_max']:
                filtered_festivals1.append(row)
        except ValueError:
            continue  # Ignorer les lignes avec des coordonnées mal formatées
    else:
        continue

filtered_festivals1 = pd.DataFrame(filtered_festivals1)

# 2. Vérifier les dates (en ignorant l'année)
# Correspondance entre les saisons et les plages de dates (mois et jours uniquement)
seasons = {
    "Avant-saison (1er janvier - 20 juin)": ("01-01", "06-20"),
    "Saison (21 juin - 5 septembre)": ("06-21", "09-05"),
    "Après-saison (6 septembre - 31 décembre)": ("09-06", "12-31")
    }

for _, row in filtered_festivals1.iterrows():
    if row['Période principale de déroulement du festival'] in seasons:
        season_start, season_end = seasons[row['Période principale de déroulement du festival']]
    
        # Convertir les dates utilisateur (ignorer l'année)
        user_start = pd.to_datetime(user_data['dates'][0]).strftime("%m-%d")
        user_end = pd.to_datetime(user_data['dates'][1]).strftime("%m-%d")
    
        # Vérifier l'intersection entre les plages de dates
        # Convertir les dates de saison et utilisateur en plages de date
        season_range = pd.date_range(f"2024-{season_start}", f"2024-{season_end}")
        user_range = pd.date_range(f"2024-{user_start}", f"2024-{user_end}")

        # Vérifie si au moins un jour des plages utilisateur et saison se chevauchent
        if any(day in season_range for day in user_range):
            filtered_festivals2.append(row)
    else:
        continue

filtered_festivals2 = pd.DataFrame(filtered_festivals2)

# 3. Vérifier les types et sous-catégories
for _, row in df_a_filtrer.iterrows():
    festival_categories = [cat.strip() for cat in row['Discipline dominante'].split(';')]  # Extraire les catégories du festival

    Categories = {
        'Musique': ['Nouvelles sous-catégories musique', 'genres musique'], 
        'Spectacle vivant': ['Nouvelles sous-catégories spectacle vivant', 'genres spectacles vivants'], 
        'Arts visuels et numériques': ['Nouvelles sous-catégories arts visuels', 'genres arts visuels et numériques'], 
        'Cinéma et audiovisuel': ['Nouvelles sous-catégories cinéma et audiovisuel', 'genres cinéma et audiovisuel'], 
        'Livre et littérature': ['Nouvelles sous-catégories livre et littérature', 'genres livre et littérature']
    }

    for category in Categories.keys():
        # Vérifier si la catégorie de l'utilisateur correspond à une catégorie du festival
        if category in user_data['types'] and category in festival_categories:
            sous_categories_col = row[Categories[category][0]]
            user_subcategories = user_data[Categories[category][1]]
            
            # Vérifier que sous_categories_col est valide avant d'itérer
            if sous_categories_col is not None:
                # Vérifier si au moins une sous-catégorie correspond
                for subcategory in user_subcategories:
                    if subcategory in sous_categories_col:
                        filtered_festivals3.append(row)
                        break

filtered_festivals3 = pd.DataFrame(filtered_festivals3)
print(filtered_festivals3)


               Nom du festival Envergure territoriale  \
0             Festival andalou              Régionale   
3                    Pharaonic                    NaN   
4              Sarcus Festival                    NaN   
10                  Ai que Bom                    NaN   
14        Métis Plaine Commune                    NaN   
...                        ...                    ...   
7209          Positiv Festival                    NaN   
7218                 Tradenvie                    NaN   
7231  Le Chant de l'Eucalyptus                    NaN   
7237                Taol Kurun                    NaN   
7266      Festival des Balkans                    NaN   

     Région principale de déroulement Département principal de déroulement  \
0          Provence-Alpes-Côte d'Azur                             Vaucluse   
3                Auvergne-Rhône-Alpes                               Savoie   
4                 Centre-Val de Loire                       Indre-et-Loire   
10 

## III. Classer et limiter les festivals

In [31]:
# Coordonnées de référence
coordonnees = user_data['coordinates']

# Calculer la distance et ajouter une colonne
def calculate_distance(geo_string, reference_coords):
    try:
        lat, lon = map(float, geo_string.split(','))
        return geodesic(reference_coords, (lat, lon)).km
    except (ValueError, AttributeError):
        return None  # Retourne None en cas d'erreur de parsing

filtered_festivals3['Distance'] = filtered_festivals3['Géocodage xy'].apply(lambda geo: calculate_distance(geo, coordonnees))

# Classement par distance
filtered_df_sorted = filtered_festivals3.sort_values(by='Distance', ascending=True)

# On garde les 3 plus proches
df_end = filtered_df_sorted.head(3)

print(df_end)

# Exporter df_end dans un fichier CSV
df_end.to_csv('df_end.csv', index=False)

               Nom du festival Envergure territoriale  \
0             Festival andalou              Régionale   
2517                Green fest                    NaN   
5766  Là ! C'est de la musique                    NaN   

     Région principale de déroulement Département principal de déroulement  \
0          Provence-Alpes-Côte d'Azur                             Vaucluse   
2517       Provence-Alpes-Côte d'Azur                             Vaucluse   
5766       Provence-Alpes-Côte d'Azur                             Vaucluse   

     Commune principale de déroulement        Site internet du festival  \
0                              Avignon   https://lefestivalandalou.com/   
2517                           Avignon  https://www.inooveproduction.fr   
5766                           Avignon        www.lacestdelamusique.com   

                     Adresse e-mail Discipline dominante  \
0     contact@lefestivalandalou.com              Musique   
2517                teddy@inoove.fr  